In [1]:
import os
import time
import pandas as pd
from github import Github
from credentials import username, password
from data_API import dataAPI

In [25]:
g = Github(username(), password())  # credentials for github login
repos = g.get_user().get_repos()  # iterable; all the repos associated with the profile incl. organizations

In [19]:
test = repos

In [22]:
for i in test:
    print(i.name)
    if i.name == "2is50-2019-2020-homework-assignment-2-pair-20":
        pair20 = i

2is50-2019-2020-homework-assignment-0-20174095
2is50-2019-2020-homework-assignment-0-20183839
2is50-2019-2020-homework-assignment-0-20191177
2is50-2019-2020-homework-assignment-0-20192968
2is50-2019-2020-homework-assignment-0-20193289
2is50-2019-2020-homework-assignment-0-2958Claudia
2is50-2019-2020-homework-assignment-0-AER-Stud
2is50-2019-2020-homework-assignment-0-alex-molnar
2is50-2019-2020-homework-assignment-0-annamoerman
2is50-2019-2020-homework-assignment-0-AnnoYingYing
2is50-2019-2020-homework-assignment-0-Arendstudent
2is50-2019-2020-homework-assignment-0-AugusteLef
2is50-2019-2020-homework-assignment-0-Awesam-one
2is50-2019-2020-homework-assignment-0-basschalkwijk
2is50-2019-2020-homework-assignment-0-benGmaker
2is50-2019-2020-homework-assignment-0-benjivdbrand
2is50-2019-2020-homework-assignment-0-BibiFloris
2is50-2019-2020-homework-assignment-0-bonjasper
2is50-2019-2020-homework-assignment-0-Br1tt21
2is50-2019-2020-homework-assignment-0-brambolinie256
2is50-2019-2020-homew

In [23]:
pair20

Repository(full_name="2IS50/2is50-2019-2020-homework-assignment-2-pair-20")

In [26]:
start = time.time()
api_data = dataAPI(repos, '2is50-2019-2020-homework-assignment-2-pair-')
end = time.time()
print("Time to complete:", end - start)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
Time to complete: 851.3139622211456


In [44]:
df_data = pd.DataFrame(api_data)
all_unique_pairs = df_data['Pair'].unique()

In [45]:
grouped_data = df_data.groupby(by=['Pair','Date','Author']).describe()

x = grouped_data['Additions'][['max']].reset_index()
x['Deletions'] = grouped_data['Deletions'][['max']].reset_index()['max']
x = x.rename(columns={'max': 'Additions'})
grouped_data = x

In [46]:
aggregated_list = []

for pair in all_unique_pairs:

    df_pair = grouped_data[grouped_data['Pair'] == pair]

    members = df_pair['Author'].unique()

    if len(members) == 2: # best scenario

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]
            m_1 = df_pair_date[df_pair_date['Author'] == members[0]]
            m_2 = df_pair_date[df_pair_date['Author'] == members[1]]

            m1_additions = df_pair_date[df_pair_date['Author'] == members[0]]['Additions'].sum()
            m2_additions = df_pair_date[df_pair_date['Author'] == members[1]]['Additions'].sum()

            m1_deletions = df_pair_date[df_pair_date['Author'] == members[0]]['Deletions'].sum()
            m2_deletions = df_pair_date[df_pair_date['Author'] == members[1]]['Deletions'].sum()


            aggregated_list.append([pair, date, m1_additions, m2_additions, m1_deletions, m2_deletions])

    elif len(members) > 2: 

        custom_more = []

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]

            m1_additions = df_pair_date[df_pair_date['Author'] == members[0]]['Additions'].sum()
            m2_additions = df_pair_date[df_pair_date['Author'] == members[1]]['Additions'].sum()
            m2_additions += df_pair_date[df_pair_date['Author'] == members[2]]['Additions'].sum()


            m1_deletions = df_pair_date[df_pair_date['Author'] == members[0]]['Deletions'].sum()
            m2_deletions = df_pair_date[df_pair_date['Author'] == members[1]]['Deletions'].sum()
            m2_deletions += df_pair_date[df_pair_date['Author'] == members[2]]['Deletions'].sum()

            custom_more.append([pair, date, m1_additions, m2_additions, m1_deletions, m2_deletions])

        aggregated_list += custom_more 

    elif len(members) < 2: 

        custom_less = []

        for date in df_pair['Date'].unique():

            df_pair_date = df_pair[df_pair['Date'] == date]

            m1_additions = df_pair_date[df_pair_date['Author'] == members[0]]['Additions'].sum()
            m2_additions = 0

            m1_deletions = df_pair_date[df_pair_date['Author'] == members[0]]['Deletions'].sum()
            m2_deletions = 0

            custom_less.append([pair, date, m1_additions, m2_additions, m1_deletions, m2_deletions])

        aggregated_list += custom_less

In [47]:
cols = ['Pair', 'Date', 'M1_Additions', 'M2_Additions', 'M1_deletions', 'M2_deletions']
df_aggregated_member = pd.DataFrame(aggregated_list, columns=cols).sort_values(by='Pair')

In [48]:
df_total_size = df_aggregated_member#[['Pair', 'Date']]
df_total_size['Member_1'] = df_aggregated_member['M1_Additions'] + df_aggregated_member['M1_deletions']
df_total_size['Member_2'] = df_aggregated_member['M2_Additions'] + df_aggregated_member['M2_deletions']
df_total_size['Total'] = df_total_size['Member_1'] + df_total_size['Member_2']
df_total_size.to_csv("../data/commit_sizes_ha2.csv", index=False) 